In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
import time
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet

from sklearn.linear_model import RidgeCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import pyreadr

## load scenarios

In [4]:
scenarios = pd.read_csv( "semisyntheticDataScenarios.txt",header =0,sep = " " )

In [5]:
scenarios

,scenario,n,p,s0,SNR,corDesign,pB,s0B
0,1,100,1000,10,0.5,low,0,0
1,2,100,1000,10,0.5,high,10,1
2,3,100,1000,10,0.5,high,10,2
3,4,100,1000,10,0.5,high,10,5
4,5,100,1000,10,1.0,low,0,0
...,...,...,...,...,...,...,...,...
571,572,300,500,40,2.0,high,10,5
572,573,300,500,40,4.0,low,0,0
573,574,300,500,40,4.0,high,10,1
574,575,300,500,40,4.0,high,10,2


In [6]:
scenarios_nosso = scenarios[scenarios["corDesign"]!="Toeplitz"]

In [7]:
scenarios_nosso["corDesign"].value_counts()

high    432
low     144
Name: corDesign, dtype: int64

### load TCGA data

In [8]:
result = pyreadr.read_r('tcgaExpression.RData')
TCGA = result['tcgaExpression']

colunas =  TCGA.columns
linhas = TCGA.index
TCGA_num = np.array(TCGA)
TCGA_num  =TCGA_num.T
TCGA = pd.DataFrame(TCGA_num)
TCGA.columns =  linhas
TCGA.index = colunas

In [9]:
TCGA.head()

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
TCGA-13-0758-01A-01R-0362-01,9.040348,5.880579,3.510885,9.519195,2.829304,6.669938,3.030365,2.935902,6.366095,2.957960,...,10.467748,10.017407,11.868558,11.786269,3.034829,2.677198,2.908470,2.912073,2.689308,2.772693
TCGA-09-0364-01A-02R-0362-01,8.852858,5.842444,3.380910,5.247880,2.816243,5.065322,3.041444,3.176795,3.253725,3.104699,...,10.491409,10.044839,11.699712,11.611780,3.016276,2.739204,2.634826,2.775122,2.722554,2.748787
TCGA-13-0723-01A-02R-0362-01,9.461118,6.034554,3.938053,9.704270,2.654948,6.449914,3.276140,2.999803,7.019574,3.061643,...,10.130387,9.618885,11.666539,11.513151,2.941443,2.519679,2.766610,2.590110,2.647876,2.603110
TCGA-13-0757-01A-01R-0362-01,8.805447,6.264489,4.676579,9.705913,2.551409,5.083082,3.227242,3.270186,4.810671,3.295613,...,10.127365,9.444374,11.365904,11.311069,3.057372,2.773288,2.547109,2.868282,2.788629,2.757732
TCGA-13-0768-01A-01R-0362-01,7.806826,6.156971,4.184898,8.007862,3.190076,6.377205,3.509807,2.974439,8.663220,2.928799,...,10.472637,9.981743,11.866901,11.808349,2.954113,2.519987,2.672115,2.643893,2.771089,2.744595


## DATA GENERATION

In [8]:
def data_generation( tcga,n, p, s0, SNR, corDesign , pB , s0B, beta_value  ):
    (N,P)= tcga.shape

    if ((s0B!=0 and pB==0) or (s0B==0 and pB!=0)):
        print('ERRO')
        return 1;

    if (s0B>0 and s0/s0B*pB>p): 
        print('ERRO')
        return 1;

    genes = np.random.choice(range(0,P), size=p, replace=False)
    tcga = tcga.iloc[:,genes]
    XR = tcga.copy()

    ######## INDEPENDENCE
    if(s0B == 0):
        signals_pos = np.random.choice(range(0,p), size=s0, replace=False)

    ########## CORRELATED
    if(s0B>0):
        nblock = int(s0/s0B)
        xx = XR.copy()
        cor_xx = np.corrcoef(np.array(XR),rowvar=False)
        rest_variables = np.arange(p)

        #### creating the blocks 
        blocks = np.zeros((nblock,pB))
        for block_iter in range(nblock):
            tamanho_new = cor_xx.shape[0] 
            cor_xx_temp = abs(cor_xx); cor_xx_temp = cor_xx_temp - np.eye(tamanho_new)

            ##find the largest cor
            sub_core = np.where(cor_xx_temp == cor_xx_temp.max())[0][0]
            
            #puts in the order the most correlated values
            ordenado = sorted(cor_xx[sub_core])
            sub_selected_variables  = np.zeros(pB)
            ##### finding the sub_index of the most correlated
            for i in range(pB):
                sub_selected_variables[i] = np.where(cor_xx[sub_core] == ordenado[tamanho_new-i-1])[0][0] 
            #take the original index of the selected variables
            selected_variables = rest_variables[sub_selected_variables.astype(int)]
            
            #saves the index in this matrix
            blocks[block_iter] = selected_variables
            
            #take out the values that have been used
            rest_variables = np.setdiff1d(rest_variables,selected_variables, assume_unique=True)
            
            #take from the cor matrix only the index that have been used
            tamanho = np.arange(cor_xx.shape[0])
            var_sobraram = np.setdiff1d(tamanho, sub_selected_variables.astype(int))
            cor_xx=cor_xx[var_sobraram,:]
            cor_xx = cor_xx[:,var_sobraram]
            
        ###choosing the signals of each block
        signal_positions = np.zeros(s0)
        for i in range(nblock):
            signal_positions[i*s0B:(i+1)*s0B]= blocks[i,0:s0B]
        signal_positions = signal_positions.astype(int)
        if(sum(signal_positions==0)>=2): 
            print("ALGO ERRADO COM OS SINAIS")
            return 1;
    ## Chosing the rows
    row_idx = np.random.choice(range(0,N), size=n, replace=False)
    test_row_idx = np.setdiff1d(np.arange(N),row_idx, assume_unique=True)

    ###getting the actual data
    Xr=XR.iloc[row_idx,:];
    Xr_test=XR.iloc[test_row_idx,:]
    n_test = Xr_test.shape[0]
    Xr.index = np.arange(n)
    ##XR
    Xr = np.array(Xr)
    Xr_test = np.array(Xr_test)
    for i in range(Xr.shape[1]):
        temp=Xr[:,i]
        temp=(temp-(temp.mean()))/np.std(temp)
        Xr[:,i]=temp
    for i in range(Xr_test.shape[1]):
        temp=Xr_test[:,i]
        temp=(temp-(temp.mean()))/np.std(temp)
        Xr_test[:,i]=temp
        
    beta0 = np.zeros(p)
    beta0[signal_positions]= beta_value

    Y0 = Xr@beta0
    Y0_test = Xr_test@beta0

    sigmae = float(((beta0@(Xr.T)@Xr@(beta0.T))/(n*SNR**2))**(1/2))
    e_training = np.random.normal(0, sigmae, n)
    e_test = np.random.normal(0, sigmae, n_test)

    Y = Y0 + e_training
    Y = Y - Y.mean()
    Y_test = Y0_test + e_test
    Y_test = Y_test - np.mean(Y_test)

    return(Xr,Xr_test,Y, Y_test, beta0, sigmae, 50/(p-s0))


In [253]:
(Xr,Xr_test,Y, Y_test, beta0, sigmae, xlim) = data_generation(TCGA,n = 20,p=20, s0=4, SNR=2, corDesign = 'High' , pB = 5 , s0B = 2, beta_value = 3 )
    

In [10]:
def Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim):
######### founding the alphas necessaries with ten-fold cross validation
    cv = ElasticNetCV(l1_ratio = 1, eps = 0.001, n_alphas=100,cv =10,max_iter=5000 ).fit(X,Y)
######## founding idx of the best alpha
    alphas_min_idx = int(np.where(cv.alphas_==cv.alpha_)[0]) 
    if alphas_min_idx <= 25:
         alphas = cv.alphas_[0:25]
    else:
        alphas = cv.alphas_[1:alphas_min_idx+1] ############### tanking only the alphas biggers than the best alpha untill null solutio
        
    solution_path = np.zeros((p,len(alphas)))
    i=0
########## creating a matrix (p x num_alphas) with the beta coefficients of each regression with each alpha
    for temp in alphas:
        regression =  ElasticNet( l1_ratio=1.0, alpha = temp, max_iter= 5000 ).fit(X, Y)
        solution_path[:,i] = regression.coef_ 
        i = i+1
    beta_cv = cv.coef_ ########## best alpha
############## Variable selection ################
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i,j in zip(beta_cv!=0,beta0!=0):
        if(i==True and j==True):
            tp = tp+1
    for i,j in zip(beta_cv!=0,beta0==0):
        if(i==True and j==True):
            fp = fp+1
    for i,j in zip(beta_cv==0,beta0==0):
        if(i==True and j==True):
            tn = tn+1
    for i,j in zip(beta_cv==0,beta0!=0):
        if(i==True and j==True):
            fn = fn+1
    if((tp+fp)==0):
        mcc=0;ppv=0
    if((tp+fp)!=0):
        mcc= ((tp*tn-fp*fn)/p)/np.sqrt((tp+fp)*(tp+fn)/p**2*(tn+fp)*(tn+fn)); ppv=tp/(tp+fp)
    tpr=tp/(tp+fn)

    rmse = np.sqrt(((Y_test-X_test@beta_cv-(Y_test.mean()))**2).mean())
    ###rmse = np.sqrt(sum((Y_test - regression.predict(X_test))**2))

##### VARIABLE RANKING #### taking the larger alpha for each predictor, that not results in a null solution for this predictor
    score = np.zeros(p)
    for var_idx in range(0,p):
        
        if((sum((solution_path[var_idx,:]!=0))==0)):  #### if the predictor is null for every alpha
            score[var_idx] = 0
        if(sum((solution_path[var_idx,:]==0))==0):    #### if the predictor is not null for every alpha
            score[var_idx]=alphas[0]
        if(sum( ((solution_path[var_idx,:]!=0))!=0 ) and  (sum((solution_path[var_idx,:]==0))!=0 )): #### else
            idx = sum(solution_path[var_idx,:]==0)  ##### idx of the first alhpa with a nonnull response for this predictor
            if (idx != len(alphas)):
                score[var_idx]=alphas[idx] 
        
    beta_pauc = np.zeros(p)
    for temp,i in zip(beta0,range(p)):
        if (temp !=0):
            beta_pauc[i] = 1
    
    pauc = roc_auc_score(beta_pauc,score, max_fpr = xlim)
    
    #return mcc, ppv, tpr, rmse, pauc, cv.coef_,score
    return mcc, ppv ,tpr , rmse, pauc   
    

In [9]:
def Henet_func(X,Y, X_test, Y_test,n,p,beta0, xlim):
######### founding the alphas necessaries with ten-fold cross validation
    cv = ElasticNetCV(l1_ratio = 0.3, eps = 0.001, n_alphas=100,cv =10,max_iter= 5000).fit(X,Y)
######## founding idx of the best alpha
    alphas_min_idx = int(np.where(cv.alphas_==cv.alpha_)[0]) 
    if alphas_min_idx <= 25:
         alphas = cv.alphas_[0:25]
    else:
        alphas = cv.alphas_[1:alphas_min_idx+1] ############### tanking only the alphas biggers than the best alpha untill null solutio
        
    solution_path = np.zeros((p,len(alphas)))
    i=0
########## creating a matrix (p x num_alphas) with the beta coefficients of each regression with each alpha
    for temp in alphas:
        regression =  ElasticNet( l1_ratio=0.3, alpha = temp, max_iter= 5000).fit(X, Y)
        solution_path[:,i] = regression.coef_ 
        i = i+1
    beta_cv = cv.coef_ ########## best alpha
############## Variable selection ################
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i,j in zip(beta_cv!=0,beta0!=0):
        if(i==True and j==True):
            tp = tp+1
    for i,j in zip(beta_cv!=0,beta0==0):
        if(i==True and j==True):
            fp = fp+1
    for i,j in zip(beta_cv==0,beta0==0):
        if(i==True and j==True):
            tn = tn+1
    for i,j in zip(beta_cv==0,beta0!=0):
        if(i==True and j==True):
            fn = fn+1
    if((tp+fp)==0):
        mcc=0;ppv=0
    if((tp+fp)!=0):
        mcc= ((tp*tn-fp*fn)/p)/np.sqrt((tp+fp)*(tp+fn)/p**2*(tn+fp)*(tn+fn)); ppv=tp/(tp+fp)
    tpr=tp/(tp+fn)

#### rmse=sqrt(mean((Y_test-X_test%*%beta.cv-mean(Y_test))^2))  ####
    ###rmse = np.sqrt(sum((Y_test - regression.predict(X_test))**2))
    rmse = np.sqrt(((Y_test-X_test@beta_cv-(Y_test.mean()))**2).mean())
##### VARIABLE RANKING #### taking the larger alpha for each predictor, that not results in a null solution for this predictor
    score = np.zeros(p)
    for var_idx in range(0,p):
        
        if((sum((solution_path[var_idx,:]!=0))==0)):  #### if the predictor is null for every alpha
            score[var_idx] = 0
        if(sum((solution_path[var_idx,:]==0))==0):    #### if the predictor is not null for every alpha
            score[var_idx] = score[var_idx]=alphas[0]
        if(sum( ((solution_path[var_idx,:]!=0))!=0 ) and  (sum((solution_path[var_idx,:]==0))!=0 )): #### else
            idx = sum(solution_path[var_idx,:]==0)  ##### idx of the first alhpa with a nonnull response for this predictor
            if (idx != len(alphas)):
                score[var_idx]=alphas[idx] 
    
    beta_pauc = np.zeros(p)
    for temp,i in zip(beta0,range(p)):
        if (temp !=0):
            beta_pauc[i] = 1
    
    pauc = roc_auc_score(beta_pauc,score, max_fpr = xlim)
    
    return mcc, ppv, tpr, rmse, pauc


In [21]:
def Ridge_func(X,Y, X_test, Y_test,n,p,beta0, xlim):
######### the alphas necessaries with ten-fold cross validation
    lambdas = np.logspace(-2,2.7, num = 150, endpoint = True)
    ######## founding idx of the best alpha
    cv = RidgeCV(alphas = lambdas, cv =10).fit(X,Y)
    beta_cv = cv.coef_

    #### rmse=sqrt(mean((Y_test-X_test%*%beta.cv-mean(Y_test))^2))  ####
    rmse = np.sqrt(((Y_test - cv.predict(X_test))**2).mean())

    ##### VARIABLE RANKING #### using the betas of the best regression model
    beta_pauc = np.zeros(p)
    for temp,i in zip(beta0,range(p)):
        if (temp !=0):
            beta_pauc[i] = 1

    pauc = roc_auc_score(beta0,abs(beta_cv), max_fpr = xlim)
    
    return rmse, pauc

In [22]:
###################333 função stability ###############
def stability(X,Y, size,steps, nlambdas):
    n, p = X.shape

    selected_variables = np.zeros((p,steps))
    #stability_scores = np.zeros((p, nlambdas))
    
    cv = ElasticNetCV(l1_ratio = 1, eps = 0.001, n_alphas=nlambdas,cv =10, max_iter =5000).fit(X,Y)
    
    alphas = cv.alphas_
    
    ##### tirando os alphas
    
    alphas_min_idx = int(np.where(cv.alphas_==cv.alpha_)[0]) 
    
    if alphas_min_idx <= 25:
         alphas = cv.alphas_[0:25]
    else:
        alphas = cv.alphas_[1:alphas_min_idx+1] ############### tanking only the alphas biggers than the best alpha untill null solutio
    stability_scores = np.zeros((p, len(alphas)))
    
    ################
    
    for idx, alpha, in enumerate(alphas):
    # This is the sampling step, where bootstrap samples are generated
    # and the structure learner is fitted
        for iteration in range(steps):
            bootstrap = np.random.choice(np.arange(n), size= m.ceil(n*size),replace=False)

            X_train = X[bootstrap, :]
            Y_train = Y[bootstrap]

      # Assume scikit-learn implementation
            regression =  ElasticNet( l1_ratio=1.0, alpha = alpha, max_iter = 5000 ).fit(X_train, Y_train)
       
            selected_variables[:, iteration] = (np.abs(regression.coef_) > 1e-4)

    # This is the scoring step, where the final stability
    # scores are computed
        stability_scores[:, idx] = selected_variables.mean(axis=1)

    return stability_scores , alphas

In [23]:
######### pegando os dados e pá
def stability_func(X,Y,size,steps,nlambdas,beta0, xlim, n, p):
    
    (scores, alphas) = stability(X,Y,size, steps, nlambdas)
    
    beta_full = scores.max(axis=1)
    beta_cv = np.array(list(beta_full))
    beta_cv[beta_cv<0.6]=0
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i,j in zip(beta_cv!=0,beta0!=0):
        if(i==True and j==True):
            tp = tp+1
    for i,j in zip(beta_cv!=0,beta0==0):
        if(i==True and j==True):
            fp = fp+1
    for i,j in zip(beta_cv==0,beta0==0):
        if(i==True and j==True):
            tn = tn+1
    for i,j in zip(beta_cv==0,beta0!=0):
        if(i==True and j==True):
            fn = fn+1
    if((tp+fp)==0):
        mcc=0;ppv=0
    if((tp+fp)!=0):
        mcc= ((tp*tn-fp*fn)/p)/np.sqrt((tp+fp)*(tp+fn)/p**2*(tn+fp)*(tn+fn)); ppv=tp/(tp+fp)
    tpr=tp/(tp+fn)
    
    beta_pauc = np.zeros(p)
    for temp,i in zip(beta0,range(p)):
        if (temp !=0):
            beta_pauc[i] = 1
    
    pauc = roc_auc_score(beta_pauc,beta_full, max_fpr = xlim)
    
    return mcc, ppv, tpr, pauc

In [21]:
beta_value = 3
n_test = 500
size = 0.632
steps = 50
nlambdas = 100

# data generation error t-student 

In [11]:
def data_generation_student( tcga,n, p, s0, SNR, corDesign , pB , s0B, beta_value, df ):
    (N,P)= tcga.shape

    if ((s0B!=0 and pB==0) or (s0B==0 and pB!=0)):
        print('ERRO')
        return 1;

    if (s0B>0 and s0/s0B*pB>p): 
        print('ERRO')
        return 1;

    genes = np.random.choice(range(0,P), size=p, replace=False)
    tcga = tcga.iloc[:,genes]
    XR = tcga.copy()

    ######## INDEPENDENCE
    if(s0B == 0):
        signals_pos = np.random.choice(range(0,p), size=s0, replace=False)

    ########## CORRELATED
    if(s0B>0):
        nblock = int(s0/s0B)
        xx = XR.copy()
        cor_xx = np.corrcoef(np.array(XR),rowvar=False)
        rest_variables = np.arange(p)

        #### creating the blocks 
        blocks = np.zeros((nblock,pB))
        for block_iter in range(nblock):
            tamanho_new = cor_xx.shape[0] 
            cor_xx_temp = abs(cor_xx); cor_xx_temp = cor_xx_temp - np.eye(tamanho_new)

            ##find the largest cor
            sub_core = np.where(cor_xx_temp == cor_xx_temp.max())[0][0]
            
            #puts in the order the most correlated values
            ordenado = sorted(cor_xx[sub_core])
            sub_selected_variables  = np.zeros(pB)
            ##### finding the sub_index of the most correlated
            for i in range(pB):
                sub_selected_variables[i] = np.where(cor_xx[sub_core] == ordenado[tamanho_new-i-1])[0][0] 
            #take the original index of the selected variables
            selected_variables = rest_variables[sub_selected_variables.astype(int)]
            
            #saves the index in this matrix
            blocks[block_iter] = selected_variables
            
            #take out the values that have been used
            rest_variables = np.setdiff1d(rest_variables,selected_variables, assume_unique=True)
            
            #take from the cor matrix only the index that have been used
            tamanho = np.arange(cor_xx.shape[0])
            var_sobraram = np.setdiff1d(tamanho, sub_selected_variables.astype(int))
            cor_xx=cor_xx[var_sobraram,:]
            cor_xx = cor_xx[:,var_sobraram]
            
        ###choosing the signals of each block
        signal_positions = np.zeros(s0)
        for i in range(nblock):
            signal_positions[i*s0B:(i+1)*s0B]= blocks[i,0:s0B]
        signal_positions = signal_positions.astype(int)
        if(sum(signal_positions==0)>=2): 
            print("ALGO ERRADO COM OS SINAIS")
            return 1;
    ## Chosing the rows
    row_idx = np.random.choice(range(0,N), size=n, replace=False)
    test_row_idx = np.setdiff1d(np.arange(N),row_idx, assume_unique=True)

    ###getting the actual data
    Xr=XR.iloc[row_idx,:];
    Xr_test=XR.iloc[test_row_idx,:]
    n_test = Xr_test.shape[0]
    Xr.index = np.arange(n)
    ##XR
    Xr = np.array(Xr)
    Xr_test = np.array(Xr_test)
    for i in range(Xr.shape[1]):
        temp=Xr[:,i]
        temp=(temp-(temp.mean()))/np.std(temp)
        Xr[:,i]=temp
    for i in range(Xr_test.shape[1]):
        temp=Xr_test[:,i]
        temp=(temp-(temp.mean()))/np.std(temp)
        Xr_test[:,i]=temp
        
    beta0 = np.zeros(p)
    beta0[signal_positions]= beta_value

    Y0 = Xr@beta0
    Y0_test = Xr_test@beta0

    sigmae = float(((beta0@(Xr.T)@Xr@(beta0.T))/(n*SNR**2))**(1/2))
    e_training = np.random.standard_t( df , n)*sigmae
    e_test = np.random.standard_t( df , n_test)*sigmae

    Y = Y0 + e_training
    Y = Y - Y.mean()
    Y_test = Y0_test + e_test
    Y_test = Y_test - np.mean(Y_test)

    return(Xr,Xr_test,Y, Y_test, beta0, sigmae, 50/(p-s0))


# Fig 3, 6, 9

In [12]:
scenarios_fig = scenarios.copy()
scenarios_fig =  scenarios_fig = scenarios_fig[(scenarios_fig.p == 2000)]
scenarios_fig = scenarios_fig[(scenarios_fig.SNR == 2)|(scenarios_fig.SNR == 0.5)]
scenarios_fig = scenarios_fig[(scenarios_fig.n == 300)|(scenarios_fig.n == 100)]
scenarios_fig = scenarios_fig[(scenarios_fig.s0 == 10)|(scenarios_fig.s0 == 40)]
scenarios_fig = scenarios_fig[(scenarios_fig.s0B == 1)|(scenarios_fig.s0B == 2)|(scenarios_fig.s0B == 5)]
scenarios_fig.index = range(24)

### lasso calculation t-student grau 3

In [24]:
results_fig = pd.DataFrame(np.zeros((scenarios_fig.shape[0],scenarios_fig.shape[1]+5)))
results_fig.columns = ["scenario","n","p","s0", "SNR", "corDesign"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [25]:
for i in range(scenarios_fig.shape[0]):
    scenario = scenarios_fig.iloc[i,0] ; n = scenarios_fig.iloc[i,1]; p = scenarios_fig.iloc[i,2]; 
    s0 = scenarios_fig.iloc[i,3]; SNR = scenarios_fig.iloc[i,4]; corDesign = scenarios_fig.iloc[i,5]
    pB =scenarios_fig.iloc[i,6] ; s0B = scenarios_fig.iloc[i,7]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation_student( TCGA ,n, p, s0, SNR, corDesign , pB , s0B, beta_value = 3, df = 3  )
    
        (mcc, ppv ,tpr , rmse, pauc) = Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.001, ppv = 0.003, tpr = 0.009, rmse = 33.447, pauc = 0.506
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.030, ppv = 0.053, tpr = 0.038, rmse = 50.921, pauc = 0.524
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2

20,3
20,4
20,5
20,6
20,7
20,8
20,9
20,10
20,11
20,12
20,13
20,14
20,15
20,16
20,17
20,18
20,19
20,20
20,21
20,22
20,23
20,24
20,25
20,26
20,27
20,28
20,29
20,30
20,31
20,32
20,33
20,34
20,35
20,36
20,37
20,38
20,39
20,40
20,41
20,42
20,43
20,44
20,45
20,46
20,47
20,48
20,49
20,50
20,51
20,52
20,53
20,54
20,55
20,56
20,57
20,58
20,59
20,60
20,61
20,62
20,63
mcc = 0.035, ppv = 0.068, tpr = 0.037, rmse = 119.751, pauc = 0.515
21,0
21,1
21,2
21,3
21,4
21,5
21,6
21,7
21,8
21,9
21,10
21,11
21,12
21,13
21,14
21,15
21,16
21,17
21,18
21,19
21,20
21,21
21,22
21,23
21,24
21,25
21,26
21,27
21,28
21,29
21,30
21,31
21,32
21,33
21,34
21,35
21,36
21,37
21,38
21,39
21,40
21,41
21,42
21,43
21,44
21,45
21,46
21,47
21,48
21,49
21,50
21,51
21,52
21,53
21,54
21,55
21,56
21,57
21,58
21,59
21,60
21,61
21,62
21,63
mcc = 0.166, ppv = 0.130, tpr = 0.300, rmse = 20.577, pauc = 0.565
22,0
22,1
22,2
22,3
22,4
22,5
22,6
22,7
22,8
22,9
22,10
22,11
22,12
22,13
22,14
22,15
22,16
22,17
22,18
22,19
22,20
22,21
22,22
22,2

### lasso calculation t-student grau 6

In [13]:
results_fig = pd.DataFrame(np.zeros((scenarios_fig.shape[0],scenarios_fig.shape[1]+5)))
results_fig.columns = ["scenario","n","p","s0", "SNR", "corDesign"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [14]:
for i in range(scenarios_fig.shape[0]):
    scenario = scenarios_fig.iloc[i,0] ; n = scenarios_fig.iloc[i,1]; p = scenarios_fig.iloc[i,2]; 
    s0 = scenarios_fig.iloc[i,3]; SNR = scenarios_fig.iloc[i,4]; corDesign = scenarios_fig.iloc[i,5]
    pB =scenarios_fig.iloc[i,6] ; s0B = scenarios_fig.iloc[i,7]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation_student( TCGA ,n, p, s0, SNR, corDesign , pB , s0B, beta_value = 3, df = 6  )
    
        (mcc, ppv ,tpr , rmse, pauc) = Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.010, ppv = 0.011, tpr = 0.022, rmse = 25.486, pauc = 0.509
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.039, ppv = 0.040, tpr = 0.056, rmse = 35.464, pauc = 0.532
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2

20,4
20,5
20,6
20,7
20,8
20,9
20,10
20,11
20,12
20,13
20,14
20,15
20,16
20,17
20,18
20,19
20,20
20,21
20,22
20,23
20,24
20,25
20,26
20,27
20,28
20,29
20,30
20,31
20,32
20,33
20,34
20,35
20,36
20,37
20,38
20,39
20,40
20,41
20,42
20,43
20,44
20,45
20,46
20,47
20,48
20,49
20,50
20,51
20,52
20,53
20,54
20,55
20,56
20,57
20,58
20,59
20,60
20,61
20,62
20,63
mcc = 0.078, ppv = 0.141, tpr = 0.075, rmse = 86.000, pauc = 0.531
21,0
21,1
21,2
21,3
21,4
21,5
21,6
21,7
21,8
21,9
21,10
21,11
21,12
21,13
21,14
21,15
21,16
21,17
21,18
21,19
21,20
21,21
21,22
21,23
21,24
21,25
21,26
21,27
21,28
21,29
21,30
21,31
21,32
21,33
21,34
21,35
21,36
21,37
21,38
21,39
21,40
21,41
21,42
21,43
21,44
21,45
21,46
21,47
21,48
21,49
21,50
21,51
21,52
21,53
21,54
21,55
21,56
21,57
21,58
21,59
21,60
21,61
21,62
21,63
mcc = 0.217, ppv = 0.138, tpr = 0.442, rmse = 16.979, pauc = 0.582
22,0
22,1
22,2
22,3
22,4
22,5
22,6
22,7
22,8
22,9
22,10
22,11
22,12
22,13
22,14
22,15
22,16
22,17
22,18
22,19
22,20
22,21
22,22
22,23
22,2

### lasso calculation t-student grau 9

In [23]:
results_fig = pd.DataFrame(np.zeros((scenarios_fig.shape[0],scenarios_fig.shape[1]+5)))
results_fig.columns = ["scenario","n","p","s0", "SNR", "corDesign"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [ ]:
for i in range(scenarios_fig.shape[0]):
    scenario = scenarios_fig.iloc[i,0] ; n = scenarios_fig.iloc[i,1]; p = scenarios_fig.iloc[i,2]; 
    s0 = scenarios_fig.iloc[i,3]; SNR = scenarios_fig.iloc[i,4]; corDesign = scenarios_fig.iloc[i,5]
    pB =scenarios_fig.iloc[i,6] ; s0B = scenarios_fig.iloc[i,7]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation_student( TCGA ,n, p, s0, SNR, corDesign , pB , s0B, beta_value = 3, df = 9  )
    
        (mcc, ppv ,tpr , rmse, pauc) = Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.013, ppv = 0.026, tpr = 0.022, rmse = 23.882, pauc = 0.514
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.029, ppv = 0.031, tpr = 0.042, rmse = 34.297, pauc = 0.534
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2

### lasso calculation

In [14]:
results_fig = pd.DataFrame(np.zeros((scenarios_fig.shape[0],scenarios_fig.shape[1]+5)))
results_fig.columns = ["scenario","n","p","s0", "SNR", "corDesign"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [28]:
for i in range(scenarios_fig.shape[0]):
    scenario = scenarios_fig.iloc[i,0] ; n = scenarios_fig.iloc[i,1]; p = scenarios_fig.iloc[i,2]; 
    s0 = scenarios_fig.iloc[i,3]; SNR = scenarios_fig.iloc[i,4]; corDesign = scenarios_fig.iloc[i,5]
    pB =scenarios_fig.iloc[i,6] ; s0B = scenarios_fig.iloc[i,7]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation( TCGA ,n, p, s0, SNR, corDesign , pB , s0B, beta_value = 3  )
    
        (mcc, ppv ,tpr , rmse, pauc) = Lasso_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.019, ppv = 0.042, tpr = 0.020, rmse = 21.919, pauc = 0.522
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39
1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49
1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.053, ppv = 0.055, tpr = 0.069, rmse = 29.754, pauc = 0.547
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31
2,32
2,33
2,34
2,35
2,36
2,37
2,38
2,39
2,40
2,41
2,42
2,43
2,44
2,45
2,46
2,47
2,48
2,49
2,50
2

20,4
20,5
20,6
20,7
20,8
20,9
20,10
20,11
20,12
20,13
20,14
20,15
20,16
20,17
20,18
20,19
20,20
20,21
20,22
20,23
20,24
20,25
20,26
20,27
20,28
20,29
20,30
20,31
20,32
20,33
20,34
20,35
20,36
20,37
20,38
20,39
20,40
20,41
20,42
20,43
20,44
20,45
20,46
20,47
20,48
20,49
20,50
20,51
20,52
20,53
20,54
20,55
20,56
20,57
20,58
20,59
20,60
20,61
20,62
20,63
mcc = 0.104, ppv = 0.156, tpr = 0.098, rmse = 75.123, pauc = 0.538
21,0
21,1
21,2
21,3
21,4
21,5
21,6
21,7
21,8
21,9
21,10
21,11
21,12
21,13
21,14
21,15
21,16
21,17
21,18
21,19
21,20
21,21
21,22
21,23
21,24
21,25
21,26
21,27
21,28
21,29
21,30
21,31
21,32
21,33
21,34
21,35
21,36
21,37
21,38
21,39
21,40
21,41
21,42
21,43
21,44
21,45
21,46
21,47
21,48
21,49
21,50
21,51
21,52
21,53
21,54
21,55
21,56
21,57
21,58
21,59
21,60
21,61
21,62
21,63
mcc = 0.244, ppv = 0.145, tpr = 0.518, rmse = 15.003, pauc = 0.589
22,0
22,1
22,2
22,3
22,4
22,5
22,6
22,7
22,8
22,9
22,10
22,11
22,12
22,13
22,14
22,15
22,16
22,17
22,18
22,19
22,20
22,21
22,22
22,23
22,2

## Henet calculation

In [11]:
results_fig = pd.DataFrame(np.zeros((scenarios_fig.shape[0],scenarios_fig.shape[1]+5)))
results_fig.columns = ["scenario","n","p","s0", "SNR", "corDesign"," pB", "s0B", "mcc", "ppv" ,"tpr" , "rmse", "pauc"]

In [12]:
for i in range(scenarios_fig.shape[0]):
    scenario = scenarios_fig.iloc[i,0] ; n = scenarios_fig.iloc[i,1]; p = scenarios_fig.iloc[i,2]; 
    s0 = scenarios_fig.iloc[i,3]; SNR = scenarios_fig.iloc[i,4]; corDesign = scenarios_fig.iloc[i,5]
    pB =scenarios_fig.iloc[i,6] ; s0B = scenarios_fig.iloc[i,7]
    
    results_i = pd.DataFrame(np.zeros((64,5)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation( TCGA ,n, p, s0, SNR, corDesign , pB , s0B, beta_value = 3  )
    
        (mcc, ppv ,tpr , rmse, pauc) = Henet_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , rmse, pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    rmse_mean = results_i.iloc[:,3].mean()
    pauc_mean = results_i.iloc[:,4].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, rmse = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean))
    
    results_fig.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, pB, s0B, mcc_mean, ppv_mean ,tpr_mean , rmse_mean, pauc_mean ]

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
0,10
0,11
0,12
0,13
0,14
0,15
0,16
0,17
0,18
0,19
0,20
0,21
0,22
0,23
0,24
0,25
0,26
0,27
0,28
0,29
0,30
0,31
0,32
0,33
0,34
0,35
0,36
0,37
0,38
0,39
0,40
0,41
0,42
0,43
0,44
0,45
0,46
0,47
0,48
0,49
0,50
0,51
0,52
0,53
0,54
0,55
0,56
0,57
0,58
0,59
0,60
0,61
0,62
0,63
mcc = 0.048, ppv = 0.017, tpr = 0.297, rmse = 21.203, pauc = 0.544
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14
1,15
1,16
1,17
1,18
1,19
1,20
1,21
1,22
1,23
1,24
1,25
1,26
1,27
1,28
1,29
1,30
1,31
1,32
1,33
1,34
1,35
1,36
1,37
1,38
1,39


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31.402555632490007, tolerance: 7.825097436195032
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.492993517047609, tolerance: 7.825097436195032
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.220208643515814, tolerance: 7.825097436195032
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarni

1,40
1,41
1,42
1,43
1,44
1,45
1,46
1,47
1,48
1,49


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.72226043497918, tolerance: 6.7020944066730666
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.920573682865928, tolerance: 6.7020944066730666
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.459871392123741, tolerance: 6.7020944066730666
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWar

1,50
1,51
1,52
1,53
1,54
1,55
1,56
1,57
1,58
1,59
1,60
1,61
1,62
1,63
mcc = 0.096, ppv = 0.043, tpr = 0.553, rmse = 29.285, pauc = 0.611
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
2,10
2,11
2,12
2,13
2,14
2,15
2,16
2,17
2,18
2,19
2,20
2,21
2,22
2,23
2,24
2,25
2,26
2,27
2,28
2,29
2,30
2,31


...

21,59
21,60
21,61
21,62
21,63
mcc = 0.155, ppv = 0.049, tpr = 0.879, rmse = 15.098, pauc = 0.554
22,0
22,1
22,2
22,3
22,4
22,5
22,6
22,7
22,8
22,9
22,10
22,11
22,12
22,13
22,14
22,15
22,16
22,17
22,18
22,19
22,20
22,21
22,22
22,23
22,24
22,25
22,26
22,27
22,28
22,29
22,30
22,31
22,32
22,33
22,34
22,35
22,36
22,37
22,38
22,39
22,40
22,41
22,42
22,43
22,44
22,45
22,46
22,47
22,48
22,49
22,50
22,51
22,52
22,53
22,54
22,55
22,56
22,57
22,58
22,59
22,60
22,61
22,62
22,63
mcc = 0.169, ppv = 0.050, tpr = 0.971, rmse = 19.790, pauc = 0.612
23,0
23,1
23,2
23,3
23,4
23,5
23,6
23,7
23,8
23,9
23,10
23,11
23,12
23,13
23,14
23,15


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 160.94351013094138, tolerance: 29.953534793149903
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 43.382427856185416, tolerance: 29.953534793149903
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 49.24118696391321, tolerance: 29.953534793149903
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceW

23,16
23,17
23,18
23,19
23,20
23,21
23,22
23,23
23,24
23,25
23,26
23,27
23,28
23,29
23,30
23,31
23,32
23,33
23,34
23,35
23,36
23,37
23,38
23,39


C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 147.9184954179717, tolerance: 35.31044939471347
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61.452914363437, tolerance: 35.31044939471347
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 75.70476647751758, tolerance: 35.31044939471347
  model = cd_fast.enet_coordinate_descent(
C:\Users\edusc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning:

23,40
23,41
23,42
23,43
23,44
23,45
23,46
23,47
23,48
23,49
23,50
23,51
23,52
23,53
23,54
23,55
23,56
23,57
23,58
23,59
23,60
23,61
23,62
23,63
mcc = 0.161, ppv = 0.047, tpr = 0.964, rmse = 23.093, pauc = 0.675


## ridge calculation

In [37]:
results_fig = pd.DataFrame(np.zeros((scenarios_fig.shape[0],scenarios_fig.shape[1]+2)))
results_fig.columns = ["scenario","n","p","s0", "SNR", "corDesign"," pB", "s0B", "rmse", "pauc"]

In [ ]:
for i in range(scenarios_fig.shape[0]):
    scenario = scenarios_fig.iloc[i,0] ; n = scenarios_fig.iloc[i,1]; p = scenarios_fig.iloc[i,2]; 
    s0 = scenarios_fig.iloc[i,3]; SNR = scenarios_fig.iloc[i,4]; corDesign = scenarios_fig.iloc[i,5]
    pB =scenarios_fig.iloc[i,6] ; s0B = scenarios_fig.iloc[i,7]
    
    results_i = pd.DataFrame(np.zeros((64,2)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation( TCGA ,n, p, s0, SNR, corDesign , pB , s0B, beta_value = 3  )
    
        (rmse, pauc) = Ridge_func(X,Y, X_test, Y_test,n,p,beta0, xlim)
        
        results_i.iloc[j,:] = [rmse, pauc]
    
        print("%i,%i" %(i,j))
        

    rmse_mean = results_i.iloc[:,0].mean()
    pauc_mean = results_i.iloc[:,1].mean()
    print("rmse = %.3f, pauc = %.3f" %(rmse_mean, pauc_mean))
    
    results_fig.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, pB, s0B, rmse_mean, pauc_mean ]

## stab calculation

In [10]:
beta_value = 3
n_test = 500
size = 0.632
steps = 50
nlambdas = 100

In [ ]:
results_fig = pd.DataFrame(np.zeros((scenarios_fig.shape[0],scenarios_fig.shape[1]+4)))
results_fig.columns = ["scenario","n","p","s0", "SNR", "corDesign"," pB", "s0B", "mcc", "ppv" ,"tpr" , "pauc"]

In [ ]:
for i in range(scenarios_fig.shape[0]):
    scenario = scenarios_fig.iloc[i,0] ; n = scenarios_fig.iloc[i,1]; p = scenarios_fig.iloc[i,2]; 
    s0 = scenarios_fig.iloc[i,3]; SNR = scenarios_fig.iloc[i,4]; corDesign = scenarios_fig.iloc[i,5]
    pB =scenarios_fig.iloc[i,6] ; s0B = scenarios_fig.iloc[i,7]
    
    results_i = pd.DataFrame(np.zeros((64,4)))
    #results_i.columns = [ "mcc", "ppv" ,"tpr" , "rmse", "pauc" ]
    for j in range(64):
            
        (X,X_test,Y ,Y_test,beta0, sigma, xlim) = data_generation( TCGA ,n, p, s0, SNR, corDesign , pB , s0B, beta_value = 3  )
    
        (mcc, ppv ,tpr , pauc) = stability_func(X,Y,size,steps,nlambdas,beta0, xlim, n, p)
        
        results_i.iloc[j,:] = [mcc, ppv ,tpr , pauc]
    
        print("%i,%i" %(i,j))
        
    mcc_mean = results_i.iloc[:,0].mean()
    ppv_mean = results_i.iloc[:,1].mean()
    tpr_mean = results_i.iloc[:,2].mean()
    pauc_mean = results_i.iloc[:,3].mean()
    print("mcc = %.3f, ppv = %.3f, tpr = %.3f, pauc = %.3f" %(mcc_mean, ppv_mean ,tpr_mean , pauc_mean))
    
    results_fig.iloc[i,:] = [scenario,n,p,s0, SNR, corDesign, pB, s0B, mcc_mean, ppv_mean ,tpr_mean ,pauc_mean ]

## Import planilha from CSV

In [ ]:
total_results = pd.read_excel('total_results_semi_v1.xlsx', engine='openpyxl', index_col=0, header = 0)

# export data to excel

In [ ]:
###RIDGE
for i in range(results_fig2.shape[0]):
    scenario = results_fig2.scenario[i]
    total_results.loc[scenario, "rmse_Ridge"] = results_fig2.rmse[i]
    total_results.loc[scenario, "pauc_Ridge"] = results_fig2.pauc[i]

In [ ]:
#####HENET
for i in range(results_fig3.shape[0]):
    scenario = results_fig3.scenario[i]
    total_results.loc[scenario, "mcc_Henet"] = results_fig3.mcc[i]
    total_results.loc[scenario, "ppv_Henet"] = results_fig3.ppv[i]
    total_results.loc[scenario, "tpr_Henet"] = results_fig3.tpr[i]
    total_results.loc[scenario, "rmse_Henet"] = results_fig3.rmse[i]
    total_results.loc[scenario, "pauc_Henet"] = results_fig3.pauc[i]

In [19]:
#####LASSO
for i in range(results_fig.shape[0]):
    scenario = results_fig.scenario[i]
    total_results.loc[scenario, "mcc_Lasso"] = results_fig.mcc[i]
    total_results.loc[scenario, "ppv_Lasso"] = results_fig.ppv[i]
    total_results.loc[scenario, "tpr_Lasso"] = results_fig.tpr[i]
    total_results.loc[scenario, "rmse_Lasso"] = results_fig.rmse[i]
    total_results.loc[scenario, "pauc_Lasso"] = results_fig.pauc[i]

In [ ]:
#####Stability
for i in range(results_fig2_stab.shape[0]):
    scenario =  results_fig2_stab.scenario[i]
    total_results.loc[scenario, "mcc_Stab"] =  results_fig2_stab.mcc[i]
    total_results.loc[scenario, "ppv_Stab"] =  results_fig2_stab.ppv[i]
    total_results.loc[scenario, "tpr_Stab"] =  results_fig2_stab.tpr[i]
    total_results.loc[scenario, "pauc_Stab"] =  results_fig2_stab.pauc[i]

In [22]:
total_results.to_excel('total_results_semi_consistencia_df6_v1.xlsx', index = True)

# formato planilha que guarda dados do student

In [16]:
################ Nós usamos para fazer a planilha no começo
total_results = pd.DataFrame(np.zeros((scenarios.shape[0],13)))
for i in range(scenarios.shape[0]):
    n = scenarios.iloc[i,1]; p = scenarios.iloc[i,2]; s0 = scenarios.iloc[i,3]; SNR = scenarios.iloc[i,4]
    corDesign = scenarios.iloc[i,5] ; pB = scenarios.iloc[i,6]; s0B = scenarios.iloc[i,7]
    scenario = scenarios.iloc[i,0]
    
    total_results.iloc[i,0:8] = [scenario, n, p, s0, SNR, corDesign, pB, s0B ]
total_results.columns=["scenario","n","p","s0","SNR","corDesign","pB","s0B","mcc_Lasso","ppv_Lasso","tpr_Lasso","rmse_Lasso", "pauc_Lasso"]
total_results.index = total_results.scenario